## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-17-40-33 +0000,bbc,Faisal Islam: Surprise inflation jump complica...,https://www.bbc.com/news/articles/c70xy9nv7p7o
1,2025-07-16-17-36-28 +0000,bbc,At least 20 killed in crush at US-backed GHF a...,https://www.bbc.com/news/articles/cg4rwrkdlzxo
2,2025-07-16-17-22-24 +0000,bbc,Serious questions for Tories over Afghan data ...,https://www.bbc.com/news/articles/c98w2e9leywo
3,2025-07-16-17-14-23 +0000,bbc,Lib Dems back 'pot zero' plan to cut energy bills,https://www.bbc.com/news/articles/cj4eg12wje5o
4,2025-07-16-17-08-42 +0000,bbc,Trump says it is highly unlikely he will fire ...,https://www.bbc.com/news/articles/c4geyrdprwjo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
32,trump,12
150,ukraine,8
84,china,5
48,israel,5
21,data,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
77,2025-07-15-18-54-00 +0000,world,A Day After Trump Promised New Arms for Ukrain...,https://www.wsj.com/world/russia/trump-advised...,36
21,2025-07-16-14-03-03 +0000,nyt,Republicans in Congress Shift to Backing Ukrai...,https://www.nytimes.com/2025/07/15/us/politics...,27
24,2025-07-16-13-39-23 +0000,nyt,Russia Appears Unfazed by Trump’s Ukraine War ...,https://www.nytimes.com/2025/07/16/world/europ...,27
59,2025-07-16-01-33-21 +0000,nyt,Trump’s Shift on Ukraine Is Good News for Euro...,https://www.nytimes.com/2025/07/15/world/europ...,27
78,2025-07-15-18-44-15 +0000,nyt,Rising Inflation Underscores Risks in Trump’s ...,https://www.nytimes.com/2025/07/15/us/politics...,26


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
77,36,2025-07-15-18-54-00 +0000,world,A Day After Trump Promised New Arms for Ukrain...,https://www.wsj.com/world/russia/trump-advised...
71,23,2025-07-15-20-10-21 +0000,bbc,Three key questions after Afghan data breach s...,https://www.bbc.com/news/articles/c9w12kdg5zko
56,19,2025-07-16-03-00-00 +0000,world,"Through Trial and Error, Iran Found Gaps in Is...",https://www.wsj.com/world/middle-east/iran-isr...
38,17,2025-07-16-10-35-23 +0000,nyt,South African AIDS Activist Pushes for H.I.V. ...,https://www.nytimes.com/2025/07/14/health/sout...
33,15,2025-07-16-12-20-00 +0000,world,Nvidia CEO Lavishes Praise on China in Beijing...,https://www.wsj.com/world/asia/nvidia-ceo-lavi...
20,15,2025-07-16-14-28-04 +0000,bbc,Canada's PM wants to fast-track 'nation buildi...,https://www.bbc.com/news/articles/cew0kw29lvro
0,13,2025-07-16-17-40-33 +0000,bbc,Faisal Islam: Surprise inflation jump complica...,https://www.bbc.com/news/articles/c70xy9nv7p7o
9,12,2025-07-16-16-00-11 +0000,bbc,'Sparring in space' – BBC gains rare access to...,https://www.bbc.com/news/articles/cy8k2nd7e9no
73,12,2025-07-15-20-07-28 +0000,nyt,Police Arrest Far-Right Leader After Anti-Immi...,https://www.nytimes.com/2025/07/15/world/europ...
49,11,2025-07-16-05-37-50 +0000,bbc,Driver held for hit-and-run death of world's '...,https://www.bbc.com/news/articles/c2d0yep9d37o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
